In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import requests
import lxml.html as lh
import bs4 as bs
import urllib.request

import seaborn as sns

In [5]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [6]:
import random # library for random number generation
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [7]:
#Getting the data from url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(url)
soup = bs.BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
data = pd.read_json(df[0].to_json(orient='records'))

In [8]:
data = pd.DataFrame(data)

In [9]:
data

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [10]:
data = data[data['Borough'] != 'Not assigned'].reset_index()

In [11]:
data = data.drop('index',axis=1)

In [12]:
data

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [13]:
data.shape

(103, 3)

In [14]:
geospatial_url = "https://cocl.us/Geospatial_data"
geo_data = pd.read_csv(geospatial_url)

In [15]:
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
data = data.merge(geo_data, on='Postal Code')

In [17]:
data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [18]:
data['Coordinates'] = list(zip(data['Latitude'],data['Longitude']))

In [19]:
data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Coordinates
0,M3A,North York,Parkwoods,43.753259,-79.329656,"(43.7532586, -79.3296565)"
1,M4A,North York,Victoria Village,43.725882,-79.315572,"(43.725882299999995, -79.31557159999998)"
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,"(43.6542599, -79.3606359)"
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,"(43.718517999999996, -79.46476329999999)"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,"(43.6623015, -79.3894938)"


In [20]:
map_toronto = folium.Map(location=[43.654260, -79.38949], zoom_start=10)

for lat, lng, borough, neighborhood in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [39]:
CLIENT_ID = '3SYUWYX1KQKZJJX4MOYS0LRGGD5XGTDHPHWE2IFVRCAJH3YP' # your Foursquare ID
CLIENT_SECRET = 'I3YUQP1W2APWR1P0SWME4T3PZIG43HR1GCHHF2IIYWQITU1L' # your Foursquare Secret
VERSION = '20200621'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3SYUWYX1KQKZJJX4MOYS0LRGGD5XGTDHPHWE2IFVRCAJH3YP
CLIENT_SECRET:I3YUQP1W2APWR1P0SWME4T3PZIG43HR1GCHHF2IIYWQITU1L


In [40]:
search_query = 'University'
radius = 500
print(search_query + ' .... OK!')

University .... OK!


In [41]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=3SYUWYX1KQKZJJX4MOYS0LRGGD5XGTDHPHWE2IFVRCAJH3YP&client_secret=I3YUQP1W2APWR1P0SWME4T3PZIG43HR1GCHHF2IIYWQITU1L&ll=43.666898,-79.390613&v=20200621&query=University&radius=500&limit=30'

In [42]:
results = requests.get(url).json()

In [43]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode
0,4b957655f964a520d2a334e3,University at St. Michael's College,"[{'id': '4bf58dd8d48988d1ae941735', 'name': 'U...",v-1593853308,False,St Joseph St,Bay,43.665711,-79.389909,"[{'label': 'display', 'lat': 43.66571145228362...",143,CA,Toronto,ON,Canada,"[St Joseph St (Bay), Toronto ON, Canada]",NaN
1,4d8a79ae6ce6a35de4a26142,University College Art Centre,"[{'id': '4bf58dd8d48988d1e2931735', 'name': 'A...",v-1593853308,False,NaN,King College Circle and Tower Rd,43.663962,-79.395385,"[{'label': 'display', 'lat': 43.66396220617091...",504,CA,Toronto,ON,Canada,"[King College Circle and Tower Rd, Toronto ON,...",NaN
2,4bfd97014cf820a1c0b0ecf4,University of Toronto Arts Centre,"[{'id': '4bf58dd8d48988d1e2931735', 'name': 'A...",v-1593853308,False,15 Kings College Circle,NaN,43.663605,-79.395159,"[{'label': 'display', 'lat': 43.66360538340076...",518,CA,Toronto,ON,Canada,"[15 Kings College Circle, Toronto ON, Canada]",NaN
3,52a379f0498e799278a72870,University of Toronto Student Family Housing,"[{'id': '4d954b06a243a5684965b473', 'name': 'R...",v-1593853308,False,35 Charles Street West,NaN,43.667912,-79.387297,"[{'label': 'display', 'lat': 43.66791215381838...",289,CA,Toronto,ON,Canada,"[35 Charles Street West, Toronto ON M4Y 1R6, C...",M4Y 1R6
4,4dc425a53151079ccfcf3ca0,University of Toronto English Language Program,"[{'id': '4bf58dd8d48988d1ae941735', 'name': 'U...",v-1593853308,False,252 Bloor St West,btwn St George & Bedford,43.665445,-79.392020,"[{'label': 'display', 'lat': 43.66544524836966...",197,CA,Toronto,ON,Canada,"[252 Bloor St West (btwn St George & Bedford),...",M5S 1V6


In [48]:
address = '81 St. Mary Street, Toronto, ON'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.666898 -79.390613


In [49]:
search_query = 'Japanese'
radius = 500
print(search_query + ' .... OK!')

Japanese .... OK!


In [50]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=3SYUWYX1KQKZJJX4MOYS0LRGGD5XGTDHPHWE2IFVRCAJH3YP&client_secret=I3YUQP1W2APWR1P0SWME4T3PZIG43HR1GCHHF2IIYWQITU1L&ll=43.666898,-79.390613&v=20200621&query=Japanese&radius=500&limit=30'

In [51]:
results = requests.get(url).json()

In [52]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode,location.crossStreet
0,4ada36ecf964a520132021e3,Aitas Japanese language school,"[{'id': '4bf58dd8d48988d13b941735', 'name': 'S...",v-1593853435,False,180 Bloor st west,43.668922,-79.395110,"[{'label': 'display', 'lat': 43.66892203336142...",426,CA,Toronto,ON,Canada,"[180 Bloor st west, Toronto ON, Canada]",NaN,NaN
1,4ad4c060f964a52062f720e3,Shogun Japanese Restaurant,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1593853435,False,154 Cumberland Street,43.669934,-79.393227,"[{'label': 'display', 'lat': 43.66993425275334...",398,CA,Toronto,ON,Canada,"[154 Cumberland Street, Toronto ON M5R 1A8, Ca...",M5R 1A8,NaN
2,4ad4c05ff964a5203af720e3,Yamato Japanese Restaurant,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1593853435,False,24 Bellair St.,43.670683,-79.391056,"[{'label': 'display', 'lat': 43.67068330431913...",422,CA,Toronto,ON,Canada,"[24 Bellair St., Toronto ON M5R 2C7, Canada]",M5R 2C7,NaN
3,4ea341925c5cd12b3601d654,Kenzo Japanese Noodle House,"[{'id': '55a59bace4b013909087cb24', 'name': 'R...",v-1593853435,False,522 Yonge St,43.663702,-79.384053,"[{'label': 'display', 'lat': 43.66370201195534...",636,CA,Toronto,ON,Canada,"[522 Yonge St (btwn Breadalbane & Grosvenor), ...",NaN,btwn Breadalbane & Grosvenor
4,4ae787b9f964a52020ac21e3,Tokyo Kitchen,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1593853435,False,Charles St.,43.668783,-79.385153,"[{'label': 'display', 'lat': 43.66878262373862...",487,CA,Toronto,ON,Canada,"[Charles St. (at Yonge St.), Toronto ON, Canada]",NaN,at Yonge St.


In [53]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,crossStreet,id
0,Aitas Japanese language school,School,180 Bloor st west,43.668922,-79.395110,"[{'label': 'display', 'lat': 43.66892203336142...",426,CA,Toronto,ON,Canada,"[180 Bloor st west, Toronto ON, Canada]",NaN,NaN,4ada36ecf964a520132021e3
1,Shogun Japanese Restaurant,Japanese Restaurant,154 Cumberland Street,43.669934,-79.393227,"[{'label': 'display', 'lat': 43.66993425275334...",398,CA,Toronto,ON,Canada,"[154 Cumberland Street, Toronto ON M5R 1A8, Ca...",M5R 1A8,NaN,4ad4c060f964a52062f720e3
2,Yamato Japanese Restaurant,Japanese Restaurant,24 Bellair St.,43.670683,-79.391056,"[{'label': 'display', 'lat': 43.67068330431913...",422,CA,Toronto,ON,Canada,"[24 Bellair St., Toronto ON M5R 2C7, Canada]",M5R 2C7,NaN,4ad4c05ff964a5203af720e3
3,Kenzo Japanese Noodle House,Ramen Restaurant,522 Yonge St,43.663702,-79.384053,"[{'label': 'display', 'lat': 43.66370201195534...",636,CA,Toronto,ON,Canada,"[522 Yonge St (btwn Breadalbane & Grosvenor), ...",NaN,btwn Breadalbane & Grosvenor,4ea341925c5cd12b3601d654
4,Tokyo Kitchen,Japanese Restaurant,Charles St.,43.668783,-79.385153,"[{'label': 'display', 'lat': 43.66878262373862...",487,CA,Toronto,ON,Canada,"[Charles St. (at Yonge St.), Toronto ON, Canada]",NaN,at Yonge St.,4ae787b9f964a52020ac21e3
5,OJA Restaurant,Sushi Restaurant,689 Yonge St,43.668943,-79.386241,"[{'label': 'display', 'lat': 43.6689434910527,...",419,CA,Toronto,ON,Canada,"[689 Yonge St (at Charles St), Toronto ON M4Y ...",M4Y 2B2,at Charles St,4adc9ad8f964a520ce2d21e3


In [55]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='black',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map